# Limpieza de la tabla de DIM_TIENDA_TEST

---

### Importación de librerías

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder, StandardScaler

### Importación de datos

In [4]:
ruta = "../Data/"

In [5]:
dim_tienda_test = pd.read_csv(f"{ruta}/DIM_TIENDA_TEST.csv")
dim_tienda_test

,TIENDA_ID,PLAZA_CVE,NIVELSOCIOECONOMICO_DES,ENTORNO_DES,MTS2VENTAS_NUM,PUERTASREFRIG_NUM,CAJONESESTACIONAMIENTO_NUM,LATITUD_NUM,LONGITUD_NUM,SEGMENTO_MAESTRO_DESC,LID_UBICACION_TIENDA,DATASET
0,680,1,C,Hogar,102.36,13,0,25.65488,-100.21207,Hogar Reunión,UT_DENSIDAD,TEST
1,730,1,C,Hogar,97.43,14,0,25.66315,-100.22738,Hogar Reunión,UT_DENSIDAD,TEST
2,650,1,C,Hogar,117.01,13,0,25.66404,-100.22993,Hogar Reunión,UT_DENSIDAD,TEST
3,670,1,C,Base,109.76,13,0,25.66508,-100.26338,Hogar Reunión,UT_DENSIDAD,TEST
4,800,1,C,Peatonal,0.00,0,0,25.69367,-100.21433,Parada Técnica,UT_TRAFICO_PEATONAL,TEST
...,...,...,...,...,...,...,...,...,...,...,...,...
100,720,6,C,Hogar,0.00,0,0,25.60253,-99.99317,Hogar Reunión,UT_DENSIDAD,TEST
101,130,6,CD,Base,110.06,10,0,25.36439,-100.10202,Hogar Reunión,UT_TRAFICO_VEHICULAR,TEST
102,320,6,C,Hogar,122.00,10,0,25.57214,-100.01371,Hogar Reunión,UT_TRAFICO_VEHICULAR,TEST
103,510,6,B,Hogar,0.00,13,0,25.19747,-99.83198,Hogar Reunión,UT_TRAFICO_VEHICULAR,TEST


In [6]:
venta = pd.read_csv(f"{ruta}/venta.csv")
venta

,TIENDA_ID,MES_ID,VENTA_TOTAL
0,813,202404,1042761.16
1,742,202404,604946.96
2,436,202404,2486787.81
3,732,202404,680701.78
4,282,202404,1227768.82
...,...,...,...
21376,885,202312,1157010.83
21377,793,202401,699916.91
21378,53,202401,1008108.28
21379,960,202401,1324156.07


In [7]:
meta_venta = pd.read_csv(f"{ruta}/Meta_venta.csv")
meta_venta

,ENTORNO_DES,Meta_venta
0,Base,480000
1,Hogar,490000
2,Peatonal,420000
3,Receso,516000


## Separación por los 105 IDs en la tabla de 'Venta'
---

Analizando los datos, nos dimos cuenta que los ID que estaban en 'Venta.csv' pero no en 'DIM_TIENDA.csv' son aquellos que se separaron para usarlos como TEST. Es por eso que se utilizarán esos para recrear el mismo proceso que se hizo para la limpieza de los datos de entrenamiento.

In [8]:
ids_test = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110,
            120, 130, 140, 150, 160, 170, 180, 190, 200, 210,
            220, 230, 240, 250, 260, 270, 280, 290, 300, 310,
            320, 330, 340, 350, 360, 370, 380, 390, 400, 410,
            420, 430, 440, 450, 460, 470, 480, 490, 500, 510,
            520, 530, 540, 550, 560, 570, 580, 590, 600, 610,
            620, 630, 640, 650, 660, 670, 680, 690, 700, 710,
            720, 730, 740, 750, 760, 770, 780, 790, 800, 810,
            820, 830, 840, 850, 860, 870, 880, 890, 900, 910,
            920, 930, 940, 950, 960, 970, 980, 990, 1000, 1010,
            1020, 1030, 1040, 1050]

venta = venta[venta['TIENDA_ID'].isin(ids_test)]

In [9]:
venta['TIENDA_ID'].nunique()

105

## Merge de las tablas

---

In [10]:
df_test = venta.merge(dim_tienda_test, on = 'TIENDA_ID', how = 'left')
df_test = df_test.merge(meta_venta, on = 'ENTORNO_DES', how = 'left')

# Creación de la columna 'éxito'

df_test["EXITO"] = (df_test["VENTA_TOTAL"] >= df_test["Meta_venta"]).astype(int)

In [11]:
df_test

,TIENDA_ID,MES_ID,VENTA_TOTAL,PLAZA_CVE,NIVELSOCIOECONOMICO_DES,ENTORNO_DES,MTS2VENTAS_NUM,PUERTASREFRIG_NUM,CAJONESESTACIONAMIENTO_NUM,LATITUD_NUM,LONGITUD_NUM,SEGMENTO_MAESTRO_DESC,LID_UBICACION_TIENDA,DATASET,Meta_venta,EXITO
0,250,202404,2019799.45,5,C,Hogar,118.93,13,6,25.76580,-100.23715,Hogar Reunión,UT_DENSIDAD,TEST,490000,1
1,970,202404,959735.17,2,C,Peatonal,83.47,7,0,26.09231,-98.27756,Oficinistas,UT_TRAFICO_PEATONAL,TEST,420000,1
2,400,202404,831299.01,5,C,Hogar,112.20,13,7,25.82921,-100.11680,Hogar Reunión,UT_DENSIDAD,TEST,490000,1
3,450,202404,1208747.34,5,C,Hogar,105.24,13,9,25.81408,-100.08988,Parada Técnica,UT_DENSIDAD,TEST,490000,1
4,380,202405,1178024.41,1,C,Base,101.66,13,10,25.70768,-100.17905,Oficinistas,UT_GAS_URBANA,TEST,480000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2123,1000,202401,1029569.92,2,C,Hogar,101.82,12,10,26.01994,-98.28429,Hogar Reunión,UT_DENSIDAD,TEST,490000,1
2124,640,202401,668567.17,5,C,Hogar,109.90,13,0,25.78546,-100.18814,Hogar Reunión,UT_DENSIDAD,TEST,490000,1
2125,850,202401,6505.79,4,C,Peatonal,118.65,14,0,25.69801,-100.10738,Parada Técnica,UT_TRAFICO_VEHICULAR,TEST,420000,0
2126,560,202312,1624049.08,1,C,Hogar,112.33,13,6,25.69058,-100.22989,Hogar Reunión,UT_TRAFICO_VEHICULAR,TEST,490000,1


## Análisis y procesamiento 

---

In [12]:
# Dimensión del dataset
print(f"Número de filas: {df_test.shape[0]}")
print(f"Número de columnas: {df_test.shape[1]}")

Número de filas: 2128
Número de columnas: 16


In [13]:
# Valores faltantes por columna
df_test.isna().sum()

TIENDA_ID                     0
MES_ID                        0
VENTA_TOTAL                   0
PLAZA_CVE                     0
NIVELSOCIOECONOMICO_DES       0
ENTORNO_DES                   0
MTS2VENTAS_NUM                0
PUERTASREFRIG_NUM             0
CAJONESESTACIONAMIENTO_NUM    0
LATITUD_NUM                   0
LONGITUD_NUM                  0
SEGMENTO_MAESTRO_DESC         0
LID_UBICACION_TIENDA          0
DATASET                       0
Meta_venta                    0
EXITO                         0
dtype: int64

In [14]:
# Ver cantidad de 0s que hay en columnas que no deberían tener 0s

df_test[df_test['VENTA_TOTAL'] == 0]

,TIENDA_ID,MES_ID,VENTA_TOTAL,PLAZA_CVE,NIVELSOCIOECONOMICO_DES,ENTORNO_DES,MTS2VENTAS_NUM,PUERTASREFRIG_NUM,CAJONESESTACIONAMIENTO_NUM,LATITUD_NUM,LONGITUD_NUM,SEGMENTO_MAESTRO_DESC,LID_UBICACION_TIENDA,DATASET,Meta_venta,EXITO


In [15]:
df_test[df_test['MTS2VENTAS_NUM'] == 0]

,TIENDA_ID,MES_ID,VENTA_TOTAL,PLAZA_CVE,NIVELSOCIOECONOMICO_DES,ENTORNO_DES,MTS2VENTAS_NUM,PUERTASREFRIG_NUM,CAJONESESTACIONAMIENTO_NUM,LATITUD_NUM,LONGITUD_NUM,SEGMENTO_MAESTRO_DESC,LID_UBICACION_TIENDA,DATASET,Meta_venta,EXITO
9,10,202405,669062.13,2,C,Base,0.0,0,0,26.02203,-98.27759,Hogar Reunión,UT_TRAFICO_VEHICULAR,TEST,480000,1
13,740,202412,356391.27,6,C,Hogar,0.0,0,0,25.59820,-99.99200,Hogar Reunión,UT_DENSIDAD,TEST,490000,0
19,710,202405,973969.62,4,C,Hogar,0.0,0,0,25.61985,-100.08667,Hogar Reunión,UT_DENSIDAD,TEST,490000,1
20,630,202412,348280.55,3,C,Base,0.0,0,0,25.66690,-100.36854,Oficinistas,UT_TRAFICO_VEHICULAR,TEST,480000,0
30,460,202407,1635951.92,3,C,Hogar,0.0,13,0,25.61702,-100.26720,Hogar Reunión,UT_DENSIDAD,TEST,490000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2102,590,202309,1045091.87,4,C,Hogar,0.0,13,0,25.66311,-100.09819,Hogar Reunión,UT_DENSIDAD,TEST,490000,1
2108,710,202310,593275.56,4,C,Hogar,0.0,0,0,25.61985,-100.08667,Hogar Reunión,UT_DENSIDAD,TEST,490000,1
2115,660,202311,477480.73,4,B,Base,0.0,0,0,25.64418,-100.08754,Oficinistas,UT_TRAFICO_VEHICULAR,TEST,480000,0
2120,460,202401,1533770.23,3,C,Hogar,0.0,13,0,25.61702,-100.26720,Hogar Reunión,UT_DENSIDAD,TEST,490000,1


In [16]:
df_test[df_test['MTS2VENTAS_NUM'] == 0]['TIENDA_ID'].nunique()

30

### Imputación

In [17]:
# Crear dataset único por tienda
tiendas_unicas = df_test.drop_duplicates(subset=['TIENDA_ID']).copy()

# Agregar promedio de ventas por tienda
ventas_promedio = df_test.groupby('TIENDA_ID')['VENTA_TOTAL'].mean().reset_index()
ventas_promedio.columns = ['TIENDA_ID', 'VENTA_AMT_promedio']
tiendas_unicas = tiendas_unicas.merge(ventas_promedio, on = 'TIENDA_ID', how = 'left')

df_knn = tiendas_unicas.copy()

# Encoding de categóricas
categoricas = ['PLAZA_CVE', 'NIVELSOCIOECONOMICO_DES', 'ENTORNO_DES', 'SEGMENTO_MAESTRO_DESC', 'LID_UBICACION_TIENDA']
le_dict = {}

for col in categoricas:
   le = LabelEncoder()
   df_knn[col + '_encoded'] = le.fit_transform(df_knn[col].astype(str))
   le_dict[col] = le

# Features para KNN
features_knn = ['PUERTASREFRIG_NUM', 'CAJONESESTACIONAMIENTO_NUM', 'VENTA_AMT_promedio',
               'PLAZA_CVE_encoded', 'NIVELSOCIOECONOMICO_DES_encoded', 'ENTORNO_DES_encoded', 
               'SEGMENTO_MAESTRO_DESC_encoded', 'LID_UBICACION_TIENDA_encoded']

# Matriz para KNN (incluye la columna target)
matriz_knn = df_knn[features_knn + ['MTS2VENTAS_NUM']].copy()

# Reemplazar 0s por NaN en la columna target
matriz_knn.loc[matriz_knn['MTS2VENTAS_NUM'] == 0, 'MTS2VENTAS_NUM'] = np.nan

# Escalar features numéricas
scaler = StandardScaler()
cols_numericas = ['PUERTASREFRIG_NUM', 'CAJONESESTACIONAMIENTO_NUM', 'VENTA_AMT_promedio']
matriz_knn[cols_numericas] = scaler.fit_transform(matriz_knn[cols_numericas])

# Aplicar KNN
knn_imputer = KNNImputer(n_neighbors=5, weights='distance')
matriz_imputada = knn_imputer.fit_transform(matriz_knn)

# Obtener valores imputados
df_knn['MTS2VENTAS_NUM_knn'] = matriz_imputada[:, -1]

# Identificar tiendas que necesitan imputación
tiendas_cero = df_knn[df_knn['MTS2VENTAS_NUM'] == 0].copy()

# Validación híbrida por grupos
for idx in tiendas_cero.index:
   tienda_actual = df_knn.loc[idx]
   valor_knn = tienda_actual['MTS2VENTAS_NUM_knn']
   
   # Buscar grupo similar (mismo segmento y ubicación)
   grupo = df_knn[
       (df_knn['SEGMENTO_MAESTRO_DESC'] == tienda_actual['SEGMENTO_MAESTRO_DESC']) &
       (df_knn['LID_UBICACION_TIENDA'] == tienda_actual['LID_UBICACION_TIENDA']) &
       (df_knn['MTS2VENTAS_NUM'] > 0)
   ]
   
   if len(grupo) >= 3:
       q25 = grupo['MTS2VENTAS_NUM'].quantile(0.25)
       q75 = grupo['MTS2VENTAS_NUM'].quantile(0.75)
       
       # Si KNN está fuera del rango, usar mediana del grupo
       if not (q25 <= valor_knn <= q75):
           valor_final = grupo['MTS2VENTAS_NUM'].median()
       else:
           valor_final = valor_knn
   else:
       # Si no hay suficientes datos del grupo, usar KNN
       valor_final = valor_knn
   
   df_knn.loc[idx, 'MTS2VENTAS_NUM'] = valor_final

# Diccionario de mapeo
mapeo_areas = df_knn.set_index('TIENDA_ID')['MTS2VENTAS_NUM'].to_dict()

# Aplicar a dataset completo
df_test['MTS2VENTAS_NUM'] = df_test['TIENDA_ID'].map(mapeo_areas)


In [18]:
df_test[df_test['MTS2VENTAS_NUM'] == 0]

,TIENDA_ID,MES_ID,VENTA_TOTAL,PLAZA_CVE,NIVELSOCIOECONOMICO_DES,ENTORNO_DES,MTS2VENTAS_NUM,PUERTASREFRIG_NUM,CAJONESESTACIONAMIENTO_NUM,LATITUD_NUM,LONGITUD_NUM,SEGMENTO_MAESTRO_DESC,LID_UBICACION_TIENDA,DATASET,Meta_venta,EXITO


In [19]:
df_test[df_test['PUERTASREFRIG_NUM'] == 0]

,TIENDA_ID,MES_ID,VENTA_TOTAL,PLAZA_CVE,NIVELSOCIOECONOMICO_DES,ENTORNO_DES,MTS2VENTAS_NUM,PUERTASREFRIG_NUM,CAJONESESTACIONAMIENTO_NUM,LATITUD_NUM,LONGITUD_NUM,SEGMENTO_MAESTRO_DESC,LID_UBICACION_TIENDA,DATASET,Meta_venta,EXITO
9,10,202405,669062.13,2,C,Base,108.006345,0,0,26.02203,-98.27759,Hogar Reunión,UT_TRAFICO_VEHICULAR,TEST,480000,1
13,740,202412,356391.27,6,C,Hogar,112.566915,0,0,25.59820,-99.99200,Hogar Reunión,UT_DENSIDAD,TEST,490000,0
19,710,202405,973969.62,4,C,Hogar,108.695439,0,0,25.61985,-100.08667,Hogar Reunión,UT_DENSIDAD,TEST,490000,1
20,630,202412,348280.55,3,C,Base,109.297885,0,0,25.66690,-100.36854,Oficinistas,UT_TRAFICO_VEHICULAR,TEST,480000,0
35,660,202403,550085.83,4,B,Base,103.515790,0,0,25.64418,-100.08754,Oficinistas,UT_TRAFICO_VEHICULAR,TEST,480000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2095,1040,202307,1261739.73,2,C,Hogar,104.314610,0,0,26.22708,-98.59373,Hogar Reunión,UT_TRAFICO_VEHICULAR,TEST,490000,1
2107,1030,202310,610065.25,2,C,Receso,121.300000,0,13,26.40152,-99.02246,Barrio Competido,UT_TRAFICO_PEATONAL,TEST,516000,1
2108,710,202310,593275.56,4,C,Hogar,108.695439,0,0,25.61985,-100.08667,Hogar Reunión,UT_DENSIDAD,TEST,490000,1
2115,660,202311,477480.73,4,B,Base,103.515790,0,0,25.64418,-100.08754,Oficinistas,UT_TRAFICO_VEHICULAR,TEST,480000,0


In [20]:
df_test[df_test['PUERTASREFRIG_NUM'] == 0]['TIENDA_ID'].nunique()

22

In [21]:
# Dataset único por tienda 
tiendas_unicas = df_test.drop_duplicates(subset=['TIENDA_ID']).copy()

# Agregar promedio de ventas por tienda
ventas_promedio = df_test.groupby('TIENDA_ID')['VENTA_TOTAL'].mean().reset_index()
ventas_promedio.columns = ['TIENDA_ID', 'VENTA_AMT_promedio']
tiendas_unicas = tiendas_unicas.merge(ventas_promedio, on = 'TIENDA_ID', how = 'left')

# Preparar datos para KNN
df_knn = tiendas_unicas.copy()

# Encoding de categóricas
categoricas = ['PLAZA_CVE', 'NIVELSOCIOECONOMICO_DES', 'ENTORNO_DES', 'SEGMENTO_MAESTRO_DESC', 'LID_UBICACION_TIENDA']
le_dict = {}

for col in categoricas:
    le = LabelEncoder()
    df_knn[col + '_encoded'] = le.fit_transform(df_knn[col].astype(str))
    le_dict[col] = le

# Features para KNN (ahora MTS2VENTAS_NUM es predictor, no target)
features_knn = ['MTS2VENTAS_NUM', 'CAJONESESTACIONAMIENTO_NUM', 'VENTA_AMT_promedio',
                'PLAZA_CVE_encoded', 'NIVELSOCIOECONOMICO_DES_encoded', 'ENTORNO_DES_encoded', 
                'SEGMENTO_MAESTRO_DESC_encoded', 'LID_UBICACION_TIENDA_encoded']

# Crear matriz para KNN
matriz_knn = df_knn[features_knn + ['PUERTASREFRIG_NUM']].copy()

# Reemplazar 0s por NaN en PUERTASREFRIG_NUM
matriz_knn.loc[matriz_knn['PUERTASREFRIG_NUM'] == 0, 'PUERTASREFRIG_NUM'] = np.nan

# Escalar features numéricas
scaler = StandardScaler()
cols_numericas = ['MTS2VENTAS_NUM', 'CAJONESESTACIONAMIENTO_NUM', 'VENTA_AMT_promedio']
matriz_knn[cols_numericas] = scaler.fit_transform(matriz_knn[cols_numericas])

# Aplicar KNN
knn_imputer = KNNImputer(n_neighbors=5, weights='distance')
matriz_imputada = knn_imputer.fit_transform(matriz_knn)

# Obtener valores imputados
df_knn['PUERTASREFRIG_NUM_knn'] = matriz_imputada[:, -1]

# Identificar tiendas que necesitan imputación
tiendas_cero_puertas = df_knn[df_knn['PUERTASREFRIG_NUM'] == 0].copy()

# Validación por grupos
for idx in tiendas_cero_puertas.index:
    tienda_actual = df_knn.loc[idx]
    valor_knn = tienda_actual['PUERTASREFRIG_NUM_knn']
    
    grupo = df_knn[
        (df_knn['SEGMENTO_MAESTRO_DESC'] == tienda_actual['SEGMENTO_MAESTRO_DESC']) &
        (df_knn['LID_UBICACION_TIENDA'] == tienda_actual['LID_UBICACION_TIENDA']) &
        (df_knn['PUERTASREFRIG_NUM'] > 0)
    ]
    
    if len(grupo) >= 3:
        q25 = grupo['PUERTASREFRIG_NUM'].quantile(0.25)
        q75 = grupo['PUERTASREFRIG_NUM'].quantile(0.75)
        
        if not (q25 <= valor_knn <= q75):
            valor_final = grupo['PUERTASREFRIG_NUM'].median()
        else:
            valor_final = valor_knn
    else:
        valor_final = valor_knn
    
    # Redondear a entero (las puertas son números enteros)
    df_knn.loc[idx, 'PUERTASREFRIG_NUM'] = round(valor_final)

# Crear diccionario de mapeo
mapeo_puertas = df_knn.set_index('TIENDA_ID')['PUERTASREFRIG_NUM'].to_dict()

# Aplicar a dataset completo
df_test['PUERTASREFRIG_NUM'] = df_test['TIENDA_ID'].map(mapeo_puertas)


In [22]:
df_test[df_test['PUERTASREFRIG_NUM'] == 0]

,TIENDA_ID,MES_ID,VENTA_TOTAL,PLAZA_CVE,NIVELSOCIOECONOMICO_DES,ENTORNO_DES,MTS2VENTAS_NUM,PUERTASREFRIG_NUM,CAJONESESTACIONAMIENTO_NUM,LATITUD_NUM,LONGITUD_NUM,SEGMENTO_MAESTRO_DESC,LID_UBICACION_TIENDA,DATASET,Meta_venta,EXITO


## Datos demográficos

---


In [26]:
demo = pd.read_csv(f"{ruta}/tiendas_ageb_demografia_TEST.csv")

In [27]:
# Drop de las columnas repetidas
demo = demo.drop(['PLAZA_CVE', 'NIVELSOCIOECONOMICO_DES', 'ENTORNO_DES', 'MTS2VENTAS_NUM',
       'PUERTASREFRIG_NUM', 'CAJONESESTACIONAMIENTO_NUM', 'LATITUD_NUM',
       'LONGITUD_NUM', 'SEGMENTO_MAESTRO_DESC'], axis = 1)

In [28]:
# Merge con el df pasado con la variable 'tienda_id'
df_final = df_test.merge(demo, on = 'TIENDA_ID', how = 'left')

In [29]:
df_final.isna().sum()

TIENDA_ID                     0
MES_ID                        0
VENTA_TOTAL                   0
PLAZA_CVE                     0
NIVELSOCIOECONOMICO_DES       0
ENTORNO_DES                   0
MTS2VENTAS_NUM                0
PUERTASREFRIG_NUM             0
CAJONESESTACIONAMIENTO_NUM    0
LATITUD_NUM                   0
LONGITUD_NUM                  0
SEGMENTO_MAESTRO_DESC         0
LID_UBICACION_TIENDA          0
DATASET                       0
Meta_venta                    0
EXITO                         0
POBTOT                        0
TOTHOG                        0
POCUPADA                      0
PDESOCUP                      0
P_18A24                       0
P_18A24_F                     0
P_18A24_M                     0
VPH_REFRI                     0
VPH_AUTOM                     0
VPH_PC                        0
VPH_NDACMM                    0
VPH_SINCINT                   0
P_12YMAS                      0
P_15YMAS                      0
P_15A49_F                     0
POB15_64

In [30]:
df_final.to_csv(f"{ruta}/tiendas_ageb_demografia_ventas_TEST.csv", index = False)